# 06 - Transformation Registry Basics

## 🧭 Goal

Understand how ODIBI's transformation registry works and how operations are discovered and registered.

This notebook will:
- Explore the global transformation registry
- Discover how operations like `pivot`, `unpivot`, `join`, and `sql` are registered
- List all available transformations
- Create a custom transformation and register it manually
- Understand when and how registration happens

**Estimated time:** 30 seconds

---

## 🧱 Core Concepts

**The Registry Pattern:**
```python
# ODIBI uses a global registry to store all transformations
registry = get_registry()  # Singleton
func = registry.get("unpivot")  # Lookup by name
result = func(df, ...)  # Execute
```

**When does registration happen?**
- At import time! When you `import odibi.operations.pivot`, the `@transformation` decorator runs immediately
- This registers the function in the global registry
- No manual setup needed

## 🔧 Setup

In [ ]:
# ✅ Environment Setup
import sys
import os
from pathlib import Path
import pandas as pd
import json

# Navigate to project root
project_root = Path.cwd().parent if Path.cwd().name == 'walkthroughs' else Path.cwd()
os.chdir(project_root)

# Create artifacts directory
artifacts_dir = Path('walkthroughs/.artifacts/06_registry')
artifacts_dir.mkdir(parents=True, exist_ok=True)

# Import ODIBI transformation system
from odibi.transformations import get_registry, transformation

# Import built-in operations (this triggers registration!)
import odibi.operations.pivot
import odibi.operations.unpivot
import odibi.operations.join
import odibi.operations.sql

print(f"✅ Environment ready")
print(f"📁 Artifacts: {artifacts_dir}")

## ▶️ Run: Explore the Registry

In [ ]:
# Get the global registry
registry = get_registry()

print("🔍 Global Transformation Registry\n")
print(f"Total transformations registered: {len(registry.list())}\n")

# List all registered transformations
print("📋 Available Transformations:\n")
transformations_data = []

for name in registry.list():
    func = registry.get(name)
    metadata = registry.get_metadata(name)
    
    transformations_data.append({
        "name": name,
        "version": metadata.get('version', 'unknown'),
        "category": metadata.get('category', 'uncategorized'),
        "tags": metadata.get('tags', [])
    })
    
    print(f"  • {name:15} v{metadata.get('version', '?'):8} [{metadata.get('category', 'uncategorized')}]")

# Save to JSON
with open(artifacts_dir / 'registry.json', 'w') as f:
    json.dump(transformations_data, f, indent=2)

# Save to CSV
df_registry = pd.DataFrame(transformations_data)
df_registry.to_csv(artifacts_dir / 'transformations.csv', index=False)

print(f"\n✅ Registry exported to:")
print(f"   - {artifacts_dir / 'registry.json'}")
print(f"   - {artifacts_dir / 'transformations.csv'}")

## 🔍 Inspect: Test a Transformation

In [ ]:
# Get a specific transformation
unpivot_func = registry.get("unpivot")

print("🧪 Testing 'unpivot' transformation:\n")

# Create test data
df = pd.DataFrame({
    "ID": ["A", "B"],
    "Jan": [100, 200],
    "Feb": [150, 250],
    "Mar": [120, 220]
})

print("Original data (wide format):")
print(df)

# Apply transformation
result = unpivot_func(df, id_vars="ID", var_name="Month", value_name="Sales")

print("\nTransformed data (long format):")
print(result)

print(f"\n✅ Shape changed: {df.shape} → {result.shape}")

## 🎨 Create: Register Custom Transformation

In [ ]:
# Define a custom transformation
@transformation("filter_threshold", version="1.0.0", category="filtering")
def filter_threshold(df, column, threshold, operation="greater_than"):
    """Filter DataFrame rows based on threshold."""
    if operation == "greater_than":
        return df[df[column] > threshold]
    elif operation == "less_than":
        return df[df[column] < threshold]
    elif operation == "equals":
        return df[df[column] == threshold]
    else:
        raise ValueError(f"Unknown operation: {operation}")

print("🎨 Created custom transformation: 'filter_threshold'\n")

# Verify it's registered
custom_func = registry.get("filter_threshold")
print(f"✅ Found in registry: {custom_func is not None}")

# Test it
df_test = pd.DataFrame({
    "product": ["Widget", "Gadget", "Gizmo", "Doohickey"],
    "sales": [100, 250, 75, 400]
})

print("\nOriginal data:")
print(df_test)

filtered = custom_func(df_test, column="sales", threshold=100, operation="greater_than")

print("\nFiltered data (sales > 100):")
print(filtered)

# Update registry exports
updated_data = []
for name in registry.list():
    metadata = registry.get_metadata(name)
    updated_data.append({
        "name": name,
        "version": metadata.get('version', 'unknown'),
        "category": metadata.get('category', 'uncategorized'),
        "tags": metadata.get('tags', [])
    })

with open(artifacts_dir / 'registry.json', 'w') as f:
    json.dump(updated_data, f, indent=2)

pd.DataFrame(updated_data).to_csv(artifacts_dir / 'transformations.csv', index=False)

print(f"\n✅ Registry updated with {len(updated_data)} transformations")

## ✅ Self-Check

In [ ]:
import time
start_time = time.time()

try:
    # Check artifacts exist
    assert (artifacts_dir / 'registry.json').exists(), "registry.json not found"
    assert (artifacts_dir / 'transformations.csv').exists(), "transformations.csv not found"
    
    # Load and validate JSON
    with open(artifacts_dir / 'registry.json') as f:
        data = json.load(f)
    
    assert len(data) >= 5, f"Expected at least 5 transformations, got {len(data)}"
    
    # Check required fields
    for item in data:
        assert 'name' in item, "Missing 'name' field"
        assert 'version' in item, "Missing 'version' field"
        assert 'category' in item, "Missing 'category' field"
    
    # Validate CSV matches JSON
    df_check = pd.read_csv(artifacts_dir / 'transformations.csv')
    assert len(df_check) == len(data), "CSV row count doesn't match JSON"
    
    # Check for duplicates
    names = [item['name'] for item in data]
    assert len(names) == len(set(names)), "Duplicate transformation names found"
    
    # Verify custom transformation is present
    assert 'filter_threshold' in names, "Custom transformation not registered"
    
    # Check runtime
    elapsed = time.time() - start_time
    assert elapsed < 30, f"Runtime {elapsed:.1f}s exceeds 30s budget"
    
    print("🎉 Walkthrough verified successfully!")
    print(f"⏱️  Runtime: {elapsed:.2f}s")
    print(f"📊 Transformations: {len(data)}")
    print(f"✅ All checks passed!")
    
except AssertionError as e:
    print(f"❌ Walkthrough failed: {e}")
    raise
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    raise

## 🧠 Reflection

### What You Learned

1. **Registry Pattern**: ODIBI uses a global singleton registry to store and lookup transformations
2. **Auto-Registration**: Transformations register automatically at import time via the `@transformation` decorator
3. **Metadata**: Each transformation carries version, category, and tags for organization

### Where This Fits in ODIBI

```
Pipeline Execution:
YAML → Node → registry.get("operation_name") → Execute Function → Result
               ↑
        This notebook explained this part!
```

The registry is the **heart of ODIBI's plugin architecture**. Any Python function decorated with `@transformation` becomes instantly available in YAML pipelines.

### Key Insights

- **Import = Register**: Just importing a module registers its transformations
- **Name-Based Lookup**: Operations are looked up by string name ("pivot", "unpivot", etc.)
- **Extensible**: You can add custom transformations without modifying core code

---

## ⏭ Next Steps

**Continue to:** [07_transformation_decorator_and_composition.ipynb](07_transformation_decorator_and_composition.ipynb)

Learn how the `@transformation` decorator works internally and how to compose transformations into pipelines.

**Deep dive:**
- Read `odibi/transformations/registry.py` - The registry implementation
- Read `odibi/transformations/decorators.py` - The `@transformation` decorator
- Read `odibi/operations/unpivot.py` - A simple example transformation